In [ ]:
from google.cloud.storage import Blob
from google.cloud import storage
from typing import List
import uuid
import os

def upload_source_list(paths: List) -> str:
    client = storage.Client()
    bucket = client.get_bucket(os.getenv('CONF_BUCKET'))
    key = f'config_{uuid.uuid4()}.jsonl'
    blob = Blob(key, bucket)

    blob.upload_from_string('\n'.join(paths))
    return f'gs://{bucket.name}/{key}'

## Example
# source_uri = upload_source_list([
#    'gs://cloud-ai-platform-1b863bec-7bcc-4f10-87ca-405bebd38d4d/aaa.jpg',
#    'gs://cloud-ai-platform-1b863bec-7bcc-4f10-87ca-405bebd38d4d/test.png']
# )
#
# print(source_uri)

In [ ]:
import requests
import uuid
import os


def build_request(model: str, source_uri: str, output_uri: str):
    return {
        "displayName": f"run_{uuid.uuid4()}",
        "model": model,
        "inputConfig": {
            "instancesFormat": "file-list",
            "gcsSource": {
                "uris": [source_uri]
            }
        },
        "outputConfig": {
            "predictionsFormat": "jsonl",
            "gcsDestination": {
                "outputUriPrefix": output_uri
            }
        },
        "dedicatedResources" : {
            "machineSpec" : {
              "machineType": "n1-standard-2"
            },
            "startingReplicaCount": 2
        },
        "manualBatchTuningParameters": {
            "batch_size": 64,
        }
    }

def trigger_predictions(source_uri: str) -> str:
    r = requests.post(os.getenv("PREDICTION_ENDPOINT"),
        json=build_request(os.getenv("MODEL"), source_uri, os.getenv("OUTPUT_URI")),
        headers={
            "Content-Type": "application/json; charset=utf-8",
            "Authorization": f"Bearer {os.getenv('TOKEN')}"
        }
    )
    print(r.text)
    return r.json()['name'].rsplit('/', 1)[-1]


In [ ]:
!pip install polling

In [ ]:
import requests
import polling
import os 


def check_if_completed(job_name:str):
    def _check():
        print(f'Check if completed {job_name}')
        r = requests.get(f'{os.getenv("PREDICTION_ENDPOINT")}/{job_name}', 
            headers={
                "Authorization": f"Bearer {os.getenv('TOKEN')}"
            }
        )
        resp = r.json()
        if resp['state'] == 'JOB_STATE_SUCCEEDED':
            return resp
        
        return False
    
    return _check

def wait_for_results(job_name:str, timeout=3600) -> str:
    try:
        resp = polling.poll(check_if_completed(job_name), timeout=timeout, step=10)
        results_dir = resp["outputInfo"]["gcsOutputDirectory"]
        return results_dir
    except polling.TimeoutException as ex: 
        print(ex)
        return None

## Example
# results_dir = wait_for_results('4150366123784667136')
# print(results_dir)


In [ ]:
from google.cloud import storage
import requests

def download_predictions(results_path: str):
  client = storage.Client()

  parsed_path = urlparse(results_path)

  for blob in client.list_blobs(parsed_path.netloc, prefix=parsed_path.path.lstrip("/")):
    for prediction in blob.download_as_text().splitlines():
        yield prediction

# Example
# download_predictions('gs://cloud-ai-platform-1b863bec-7bcc-4f10-87ca-405bebd38d4d/batchprediction-untitled_1618298013322_202141372930-2021-04-13T09:19:12.163334Z')

In [ ]:
!pip install shapely

In [ ]:
from google.cloud import storage
import requests
import json
import shapely.wkt
from shapely.geometry import mapping


def extract_title(id):
    name = id.rsplit('/', 1)[-1]
    return name.rsplit('_', 2)[0]

def lat_lon(meta):
    c = meta['center']
    g = shapely.wkt.loads(s)
    m = mapping(g)
    return m['coordinates']

def build_feed_request(prediction, metadata):
    p = json.loads(prediction)
    url = f'https://storage.googleapis.com/{p["ID"][5:]}'
    title = extract_title(p["ID"])
    lat, lon = lat_lon(metadata[title])
    d = title.rsplit("_", 1)[-1]
    confidence_level = p["annotations"][0]["classification"]["score"]
    return { 
        "location": "California, US",
        "url": url,
        "longitude": lat, 
        "latitude": lon, 
        "processing_date": d,
        "confidence_level": confidence_level
    }

def feed_webapp(predictions, metadata):
    for prediction in predictions:
        r = requests.post(os.getenv("WEBAPP_URL"),
            json=build_feed_request(prediction, metadata),
            headers={
                "Content-Type": "application/json; charset=utf-8"
            }
        )

# Example
# feed_webapp(download_predictions('gs://cloud-ai-platform-1b863bec-7bcc-4f10-87ca-405bebd38d4d/batchprediction-untitled_1618298013322_202141372930-2021-04-13T09:19:12.163334Z'), download_metadata())

In [ ]:
from google.cloud.storage import Blob
from google.cloud import storage
import json

def download_metadata():
    client = storage.Client()
    bucket = client.get_bucket('sentinel2-tiles')
    key = 'tiles_metadata.json'
    blob = Blob(key, bucket)

    meta = blob.download_as_string()
    
    return { t["title"]: t for t in json.loads(meta)}


In [ ]:
!pip install Pillow

In [ ]:
from google.cloud.storage import Blob
from google.cloud import storage
import json
import numpy as np
import json
from PIL import Image


def download_test():
    client = storage.Client()
    bucket = client.get_bucket('sentinel2-tiles')
    key = 'S2A_MSIL1C_20200720T183921_N0209_R070_T10SGF_20200720T221625/S2A_MSIL1C_20200720T183921_N0209_R070_T10SGF_20200720T221625_block_0.jpg'

    blob = Blob(key, bucket)

    blob.download_to_filename('/tmp/img.jpg')
    

def convert_to_json(image_file):
    """Open image, convert it to numpy and create JSON request"""
    img = Image.open(image_file)
    img_array = np.array(img)
    predict_request = img_array.tolist()
    with open('/tmp/source.jsonl', 'a') as output_file:
        json.dump(predict_request, output_file)
        output_file.write('\n')
        json.dump(predict_request, output_file)
        output_file.write('\n')

download_test()
convert_to_json('/tmp/img.jpg')



In [ ]:
data = '''{"ID":"gs://cloud-ai-platform-1b863bec-7bcc-4f10-87ca-405bebd38d4d/aaa.jpg","annotations":[{"annotation_spec_id":"8218501971951222784","classification":{"score":0.41259068},"display_name":"ui"},{"annotation_spec_id":"996979969462632448","classification":{"score":0.58740932},"display_name":"code"}]}
{"ID":"gs://cloud-ai-platform-1b863bec-7bcc-4f10-87ca-405bebd38d4d/test.png","annotations":[{"annotation_spec_id":"8218501971951222784","classification":{"score":0.86045122},"display_name":"ui"},{"annotation_spec_id":"996979969462632448","classification":{"score":0.13954881},"display_name":"code"}]}'''

In [ ]:
from google.cloud import storage

def get_paths():
    
    client = storage.Client()
    
    return [f'gs://sentinel2-tiles/{blob.name}' for blob in client.list_blobs('sentinel2-tiles', prefix='S2') if not blob.name.endswith('jp2')]


In [ ]:
token = !gcloud auth application-default print-access-token
%env CONF_BUCKET=pred-test-fire
%env PREDICTION_ENDPOINT=https://europe-west1-aiplatform.googleapis.com/v1alpha1/projects/276482548675/locations/europe-west1/batchPredictionJobs
%env TOKEN={token[-1]}
%env MODEL=projects/276482548675/locations/europe-west1/models/1511661362424578048
%env OUTPUT_URI=gs://pred-test-fire
%env WEBAPP_URL=https://forest-fires-service-v4-jrtltode5a-uw.a.run.app/api/fires/

In [ ]:
# paths = [
#     'gs://pred-test-fire/test128_block_0.jpg',
#     'gs://pred-test-fire/test128_block_1.jpg']
paths = [
    'gs://sentinel2-tiles/S2A_MSIL1C_20200720T183921_N0209_R070_T10SGF_20200720T221625/S2A_MSIL1C_20200720T183921_N0209_R070_T10SGF_20200720T221625_block_0.jpg',
    'gs://sentinel2-tiles/S2A_MSIL1C_20200720T183921_N0209_R070_T10SGF_20200720T221625/S2A_MSIL1C_20200720T183921_N0209_R070_T10SGF_20200720T221625_block_1.jpg',
]
#paths = get_paths()
meta = download_metadata()
source_uri = upload_source_list(paths) 
job_name = trigger_predictions(source_uri)
results_uri = wait_for_results(job_name)
predictions = download_predictions(results_uri)
#feed_webapp(predictions, meta)